# Menu Database Analysis and Report
### For Analytics, we should use the menu_cstore database
Because column store is optimized to run ad hoc queries on billions of rows without indexes,
combining columnar data on low-cost object storage with multi-threaded query processing
which is perfect for cloud data warehousing/analytics.

In [3]:
import pandas as pd
from IPython.display import display
import mysql.connector

In [4]:
# Functions that used to connect with the cloud database and display results.
def create_connection():
    conn = None
    try:
       conn = mysql.connector.connect(
          user="DB00004175",
          password="tTNlXEirHh)EmwXhtp4koTR",
          host="menu-data-db.mdb0002067.db.skysql.net",
          ssl_ca="skysql_chain.pem",
          port=5001)
       conn.autocommit = False
       return conn
    except mysql.connector.Error as e:
       print(f"Error connecting to MariaDB Platform: {e}")
    return conn

def sql_execute(conn, sql):

    try:
        cur = conn.cursor()
        cur.execute(sql, multi=True)
        conn.commit()
    except mysql.connector.Error as e:
        print(e)

def sql_execute_show(conn, sql):

    try:
        df = pd.read_sql(sql, con = conn)
        display(df)
        return df
    except mysql.connector.Error as e:
        print(e)

# Querying Example
- Only call the create_connection() function once
- Use the sql_execute_show() function to display results in python
- Remember to close connection at the end of your session

In [5]:
# Only call once
conn = create_connection()

In [5]:
# sql_execute_show() conn parameter = conn
# sql parameter = the string of you SQL query
sql_execute_show(conn, sql = '''
SELECT year, AVG(calories), AVG(total_fat), AVG(Protein) FROM menu_cstore.nutrition_facts
GROUP BY year
;
''')

,year,AVG(calories),AVG(total_fat),AVG(Protein)
0,2008,477.7172,26.1575,18.9965
1,2010,369.4070,17.5817,12.5405
2,2012,439.7074,32.7203,24.1088
3,2013,370.0542,16.1378,13.4993
4,2014,375.3779,16.7372,13.4871
5,2015,407.4768,18.4459,15.2967
6,2016,378.0453,16.8043,14.0246
7,2017,374.0719,16.9589,13.9133
8,2018,386.7493,17.4970,14.7468


In [16]:
sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(calories), AVG(total_fat), AVG(Protein)
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')

,r_name,AVG(calories),AVG(total_fat),AVG(Protein)
0,None,459.4577,24.4788,17.7381
1,7 Eleven,289.9115,14.1728,9.8063
2,Applebee's,535.8140,29.0960,23.8558
3,Arby's,347.4370,15.9087,13.2212
4,Auntie Anne's,275.5607,5.2865,4.4921
...,...,...,...,...
92,Whataburger,434.8670,18.6071,13.5658
93,White Castle,378.2370,13.7935,8.1544
94,Wingstop,220.3913,11.6316,12.3684
95,Yard House,697.2137,40.0058,30.8827


In [17]:
conn.close()


# MariaDB ROLLUP Syntax Example
Reference: https://mariadb.com/kb/en/select-with-rollup/

**For each restaurant in the Entrées category,
perform a ROLLUP query, then use pandas pivot function to generate a nicer table**


In [16]:
df_rollup = sql_execute_show(conn, sql = '''
SELECT
    COALESCE(b.r_name, "All Restaurants") AS "Restaurant",
    COALESCE(year, "All Years") AS "Year",
    AVG(calories), AVG(total_fat), AVG(Protein)
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
WHERE b.r_name != "None" AND a.cat_id = 5
GROUP BY b.r_name, year WITH ROLLUP
;
''')

,Restaurant,Year,AVG(calories),AVG(total_fat),AVG(Protein)
0,7 Eleven,2013,212.5000,9.6250,8.3750
1,7 Eleven,2014,213.3333,7.7778,8.2222
2,7 Eleven,2015,217.2727,8.5455,9.0909
3,7 Eleven,2016,210.0000,9.6667,14.0000
4,7 Eleven,2017,NaN,NaN,NaN
...,...,...,...,...,...
715,Zaxby's,2016,950.1818,53.7727,72.9091
716,Zaxby's,2017,957.0000,55.5789,68.3684
717,Zaxby's,2018,1085.5660,52.9811,89.4906
718,Zaxby's,All Years,974.4121,67.8794,77.1709


**For the average calories for each restaurant in the Entrées category over the years**

In [18]:
df_rollup.pivot(index=["Restaurant"], columns="Year", values="AVG(calories)")

Year,2008,2010,2012,2013,2014,2015,2016,2017,2018,All Years
Restaurant,,,,,,,,,,
7 Eleven,NaN,NaN,NaN,212.5000,213.3333,217.2727,210.0000,NaN,NaN,214.1935
All Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,674.8127
Applebee's,345.0000,NaN,888.6207,831.2069,767.9104,956.4935,755.2941,742.5532,781.9565,822.5980
Arby's,387.2857,414.0000,345.0000,335.0000,390.0000,390.0000,400.0000,387.5000,505.7143,403.2750
BJ's Restaurant & Brewhouse,NaN,NaN,NaN,NaN,NaN,852.9091,855.3061,821.2500,826.8056,838.2589
...,...,...,...,...,...,...,...,...,...,...
Whataburger,617.7778,590.5556,565.5556,684.1379,684.8276,659.0909,647.9412,656.1538,665.3659,649.7297
White Castle,NaN,NaN,460.0000,460.0000,230.0000,367.1429,1284.0000,1284.0000,280.0000,874.2857
Wingstop,NaN,NaN,NaN,NaN,NaN,NaN,278.9333,278.9333,278.9333,278.9333


**For the average protein for each restaurant in the Entrées category over the years**

In [55]:
#df_rollup.pivot(index=["r_name"], columns="year", values="AVG(Protein)")

**For the average total_fat for each restaurant in the Entrées category over the years**

In [56]:
#df_rollup.pivot(index=["r_name"], columns="year", values="AVG(total_fat)")

## We can also add the food_category in the GROUP BY clause, to make it a three dimensional cube

In [28]:
df_rollup2 = sql_execute_show(conn, sql = '''
SELECT
    COALESCE(year, "All Years") AS "Year",
    COALESCE(b.r_name, "All Restaurants") AS "Restaurant",
    COALESCE(c.cat_name, "All Categories") AS "Category",
    AVG(calories), AVG(total_fat), AVG(Protein)
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
LEFT JOIN menu_cstore.food_category c ON a.cat_id = c.cat_id
WHERE r_name != "None" AND cat_name != "None" AND r_name = "Applebee's"
GROUP BY b.r_name, c.cat_name, year WITH ROLLUP
;
''')

,Year,Restaurant,Category,AVG(calories),AVG(total_fat),AVG(Protein)
0,2008,Applebee's,Appetizers & Sides,480.0000,13.0000,NaN
1,2012,Applebee's,Appetizers & Sides,662.3864,36.6000,32.3953
2,2013,Applebee's,Appetizers & Sides,668.4524,38.0930,33.8049
3,2014,Applebee's,Appetizers & Sides,626.2500,34.2955,29.6136
4,2015,Applebee's,Appetizers & Sides,546.5152,29.0303,18.7273
...,...,...,...,...,...,...
94,2017,Applebee's,Toppings & Ingredients,168.7879,11.3333,2.2121
95,2018,Applebee's,Toppings & Ingredients,176.2162,9.9730,3.0000
96,All Years,Applebee's,Toppings & Ingredients,174.7368,13.0574,2.5368
97,All Years,Applebee's,All Categories,535.8140,29.0960,23.8558


In [29]:
df_rollup2.pivot(index=["Restaurant", "Category"], columns="Year", values="AVG(calories)")

Year                                     2008      2012      2013      2014  \
Restaurant      Category                                                      
All Restaurants All Categories            NaN       NaN       NaN       NaN   
Applebee's      All Categories            NaN       NaN       NaN       NaN   
                Appetizers & Sides      480.0  662.3864  668.4524  626.2500   
                Baked Goods               NaN  700.0000  315.0000  315.0000   
                Beverages                 NaN  114.7368  115.7895  202.2727   
                Burgers                   NaN  932.0000  949.3750  946.8750   
                Desserts                230.0  640.7692  703.0769  821.0000   
                Entrees                 345.0  888.6207  831.2069  767.9104   
                Fried Potatoes            NaN  698.0000  515.0000  670.0000   
                Pizza                     NaN  750.0000  490.0000  460.0000   
                Salads                  250.0  617.5000  588.0488  576.9444   
                Sandwiches              360.0  770.4348  740.5000  748.2353   
                Soup                    150.0  281.7647  294.7619  319.4118   
                Toppings & Ingredients    NaN  167.8125  170.9375  147.0000   

Year                                        2015      2016      2017  \
Restaurant      Category                                               
All Restaurants All Categories               NaN       NaN       NaN   
Applebee's      All Categories               NaN       NaN       NaN   
                Appetizers & Sides      546.5152  414.3750  567.0000   
                Baked Goods             315.0000       NaN       NaN   
                Beverages               201.4583  157.6623  190.2308   
                Burgers                 985.3333  910.0000  846.0000   
                Desserts                709.3333  805.0000  835.0000   
                Entrees                 956.4935  755.2941  742.5532   
                Fried Potatoes          648.3333  584.0000  560.0000   
                Pizza                   450.0000  455.0000       NaN   
                Salads                  584.5161  556.6667  733.5294   
                Sandwiches              697.0588  778.6667  716.3636   
                Soup                    298.0000  330.0000  251.4286   
                Toppings & Ingredients  245.5769  159.8214  168.7879   

Year                                        2018  All Years  
Restaurant      Category                                     
All Restaurants All Categories               NaN   535.8140  
Applebee's      All Categories               NaN   535.8140  
                Appetizers & Sides      556.8750   581.5602  
                Baked Goods                  NaN   411.2500  
                Beverages               170.2756   169.3909  
                Burgers                 852.7273   925.2000  
                Desserts                842.2222   748.6747  
                Entrees                 781.9565   822.5980  
                Fried Potatoes          420.0000   594.5161  
                Pizza                        NaN   524.4444  
                Salads                  697.7778   609.5413  
                Sandwiches              733.8462   739.8291  
                Soup                    261.4286   295.4444  
                Toppings & Ingredients  176.2162   174.7368